## 0. Load Libraries

In [1]:
import pandas as pd
import numpy as np
from surprise.model_selection import GridSearchCV

In [2]:
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering

## 1. Obtain Data

In [3]:
reviews_df = pd.read_csv('../data/interim/final_ratings.csv')

In [3]:
reviews_df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,lang,polarity
0,2539,55688172,2015-12-04,25160947,Peter,Great host,ENGLISH,0.6249
1,2539,97474898,2016-08-27,91513326,Liz,Nice room for the price. Great neighborhood. J...,ENGLISH,0.9228
2,2539,105340344,2016-10-01,90022459,Евгений,Very nice apt. New remodeled.,ENGLISH,0.4754
3,2539,133131670,2017-02-20,116165195,George,Great place to stay for a while. John is a gre...,ENGLISH,0.9231
4,2539,157777930,2017-06-04,1806142,Isaac,I really enjoyed my time here in deep south Br...,ENGLISH,0.9881


In [4]:
reviews_df.shape

(1053172, 8)

In [5]:
reviews_df.polarity.describe()

count    1.053172e+06
mean     8.133919e-01
std      2.722260e-01
min     -1.000000e+00
25%      7.906000e-01
50%      9.215000e-01
75%      9.666667e-01
max      1.000000e+00
Name: polarity, dtype: float64

## 2. Surprise SVD

### Splitting Data into training and testing

In [4]:
from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import cross_validate, train_test_split

In [5]:
reader = Reader(rating_scale=(-1,1))

In [6]:
data = Dataset.load_from_df(reviews_df[['listing_id', 'reviewer_id', 'polarity']], reader)

In [7]:
data.read_ratings

<bound method Dataset.read_ratings of <surprise.dataset.DatasetAutoFolds object at 0x10bf48860>>

In [8]:
trainset, testset = train_test_split(data, test_size=.25)

### Baseline SVD

In [11]:
algo = SVD(n_factors=10, n_epochs=10)

In [12]:
# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

In [43]:
# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.2661


0.2660984221479662

### Using SVDpp

In [13]:
from surprise import SVDpp

In [15]:
#SVD Matrix Factorization
algo = SVDpp(n_factors=10,n_epochs=10)
algo.fit(trainset)
predictions_svd = algo.test(testset)

In [16]:
accuracy.rmse(predictions_svd)

RMSE: 0.2669


0.26688308324524357

### SVD using GridSearchCV

In [54]:
param_grid = {'n_epochs': [100,200,300], 'lr_all': [0.005],
              'reg_all': [0.6],'n_factors':[50,100,150]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

In [ ]:
gs.fit(data)

In [ ]:
# best RMSE score
print("The best RMSE Score is :",gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print("Combination of parameters for best RMSE : ",gs.best_params['rmse'])

GridSearchCV on SVD run on HPC with the following results -   
```
Best RMSE: 0.2677603922989846
Best params: {'n_epochs': 50, 'n_factors': 100, 'random_state': 42, 'lr_all': 0.005, 'reg_all': 0.02}
```

### Knn

In [ ]:
#lets configure some parameters for Collaborative Filtering Algo
sim_options = {
    'name': 'pearson', #similarity measure
    'user_based': False #item based
}
#For user based ->True
#For name ->pearson,cosine,msd,pearson_baseline


In [ ]:
#this is baseline configuration for optimizing the error
bsl_options = {'method': 'als',#another option is sgd
               'n_epochs': 5, # number of iterations
               'reg_u': 12, # user-regularisation parameter
               'reg_i': 5   # item-regularisation parameter
               }

We will try KNN with maximum neighbour as 3 and minimum as 1.We train our model on trainingset and generate prediction on test set.

In [9]:
#KNN
knn = KNNBasic(sim_options=sim_options,k=3,min_k=1) #neighbours=3
knn.fit(trainset)
predictions_knn = knn.test(testset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


### NMF

In [9]:
algo = NMF()

In [10]:
algo.fit(trainset)

In [12]:
predictions_nmf = algo.test(testset)

In [13]:
accuracy.rmse(predictions_nmf)

RMSE: 0.3084


0.3083735643697148

### Comparing Algorithms using cross validation

In [ ]:
perf_results = []
for algo in [KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering(), SlopeOne(), NMF(), NormalPredictor()]:
    results = cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
    
    results_df = pd.DataFrame.from_dict(results).mean(axis=0)
    results_df = results_df.append(pd.Series([str(algo).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    perf_results.append(results_df)
    
pd.DataFrame(perf_results).set_index('Algorithm').sort_values('test_rmse')    

Above function run on HPC yielded following results -   
```
                 test_rmse    fit_time  test_time
Algorithm                                        
BaselineOnly      0.265576    7.206392   3.875109
KNNBaseline       0.266391   47.278704   4.241048
KNNWithMeans      0.272584   42.888045   5.331833
KNNWithZScore     0.272735   47.070919   4.372444
KNNBasic          0.272750   40.968443   4.460657
NMF               0.308257  127.228344   4.146235
NormalPredictor   0.350181    1.651643   3.924981
CoClustering      0.440552   87.012477   3.112529
```

## 3. Predicting top 3 recommendations

In [17]:
from collections import defaultdict
 
def get_top3_recommendations(predictions, topN = 3):
     
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
     
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]
     
    return top_recs

In [18]:
top3_recommendations = get_top3_recommendations(predictions)

In [22]:
# For each user we print 3 recommendation
i=0;
for uid, user_ratings in top3_recommendations.items():
    print(uid, [iid for (iid, _) in user_ratings])
    i=i+1;
    if(i==5):
        break; 

6800277 [117180600, 223960747, 33160735]
18402330 [66761764, 46562245, 184214926]
10404638 [10234262, 1222411, 138698041]
14805984 [98518310, 98410232, 122178507]
16157731 [37187868, 89539814, 35971773]
